In [2]:
!pip install loguru

  Using cached loguru-0.7.2-py3-none-any.whl.metadata (23 kB)
Using cached loguru-0.7.2-py3-none-any.whl (62 kB)


In [1]:
from loguru import logger

ModuleNotFoundError: No module named 'loguru'

In [ ]:
def make_api_request_data(target_data: List[Dict[str, Any]], args: Namespace) -> List[Dict[str, Any]]:
    # https://platform.openai.com/docs/guides/batch/getting-started 참조
    request_data = []

    for data_idx, datum in enumerate(tqdm(target_data, desc="Making API request data", mininterval=1)):
        request_data.append(
            {
                "custom_id": str(data_idx),  # TODO: id를 년, 월, 일, 왕 이름으로 변경
                "method": "POST",
                "url": "/v1/chat/completions",
                "body": {
                    "model": args.model,
                    "messages": [
                        {"role": "system", "content": SYSTEM_PROMPT},
                        {"role": "user", "content": datum["hanja"]},
                    ],
                    "max_tokens": args.max_completion_tokens,
                    "temperature": args.temperature,
                    "seed": args.seed,
                    "response_format": {"type": "json_object"},
                },
            }
        )

    logger.info(f"Total {len(request_data)} API requests are made!")
    return request_data